# Serverless

- Name: Aryan Ebrahimpour
- ID: 98726942

In [ ]:
#r "nuget: Plotly.NET, 2.0.0-preview.16"
#r "nuget: Plotly.NET.Interactive, 2.0.0-preview.16"
#r "nuget: FSharp.Data"
#r "nuget: NodaTime"
#r "nuget: MathNet.Numerics"
#r "nuget: MathNet.Numerics.FSharp"
#r "nuget: FSharp.SystemTextJson"

open System
open Plotly.NET
open Plotly.NET.Interactive
open NodaTime
open System.Text.Json
open System.Text.Json.Serialization

Installed Packages FSharp.Data, 4.2.8 FSharp.SystemTextJson, 0.17.4 MathNet.Numerics, 5.0.0 MathNet.Numerics.FSharp, 5.0.0 NodaTime, 3.1.0 Plotly.NET, 2.0.0-preview.16 Plotly.NET.Interactive, 2.0.0-preview.16

Loading extensions from `Plotly.NET.Interactive.dll`

In [ ]:
#load "Serverless.fsx"
open Avestura.Serverless
open MathNet.Numerics.Distributions

In [ ]:
let inline aggregateMeassure util cost respT = (util * util) / (cost * respT)

let inline simAggr (sctx: SimulatorContextBatchQoS) =
    let u, _ = sctx.utilization
    let c, _ = sctx.cost
    let r, _ = sctx.responseTime
    aggregateMeassure u c r |> float

In [ ]:
open SimulationRunnerEngine
open EvaluatorConfigs
open AllSchedulers

let iterationsCount = 10

let queueD = genQueue config_UFD_AW_N 3u
let simD = runSim queueD AllSchedulers.random240_50MeanWaitScheduler

[simD] |> SimulatorContextBatch.getBatchQoS |> (fun x -> x.responseTime)


Item1,Item2
9.873245408018017,1.900899797762011


In [ ]:
//simD.containers["container-1"] |> Container.getQosMeassures |> (fun x -> x.Value.responseTime)
simD.containers["container-1"].totalRestorationDuration

Days,NanosecondOfDay,Hours,Minutes,Seconds,Milliseconds,SubsecondTicks,SubsecondNanoseconds,BclCompatibleTicks,TotalDays,TotalHours,TotalMinutes,TotalSeconds,TotalMilliseconds,TotalTicks,TotalNanoseconds
0,562000000000,0,9,22,0,0,0,5620000000,0.006504629629629629,0.15611111111111112,9.366666666666667,562,562000,5620000000,562000000000


In [ ]:
open SimulationRunnerEngine
open EvaluatorConfigs
open AllSchedulers
open Microsoft.FSharp.Data.UnitSystems.SI.UnitNames
open Avestura.Serverless

module Sim1 =
    let iterationsCount = 4

    let queue = genQueue config_UFD_AW_N 3u
    let runForScheduler = runBatch iterationsCount queue
    //let noMer, staticSc, randSc, dynNeut, dynCtx, sandSch = forAllSchedulers runForScheduler
    //let q1, q2, q3, q4, q5, q6 = qosOf noMer, qosOf staticSc, qosOf randSc, qosOf dynNeut, qosOf dynCtx, qosOf sandSch

    let q1 = { utilization = (61.7, 23.5); responseTime = (22.5<second>, 6.1<second>); turnaroundTime = (86.7<second>, 62.8<second>); cost = (10.86<usd>, 0.0<usd>)}
    let q2 = { utilization = (51.6, 7);    responseTime = (9.0<second>, 1.7<second>);  turnaroundTime = (74.5<second>, 12.7<second>); cost = (15.73<usd>, 0.0<usd>)}
    let q3 = { utilization = (54.0, 8.5);  responseTime = (9.7<second>, 2.9<second>);  turnaroundTime = (72.5<second>, 14.3<second>); cost = (14.91<usd>, 0.0<usd>)}
    let q4 = { utilization = (66.5, 21.1); responseTime = (18.3<second>, 7.0<second>); turnaroundTime = (78.3<second>, 48.8<second>); cost = (10.65<usd>, 0.0<usd>)}
    let q5 = { utilization = (66.8, 21);   responseTime = (18.2<second>, 7.0<second>); turnaroundTime = (78.6<second>, 48.2<second>); cost = (10.62<usd>, 0.0<usd>)}
    let q6 = { utilization = (73.5, 21.2); responseTime = (10.3<second>, 1.3<second>); turnaroundTime = (73.4<second>, 63.2<second>); cost = (9.22<usd>, 0.0<usd>)}

In [ ]:
let jsonOption = JsonSerializerOptions()
jsonOption.Converters.Add(JsonFSharpConverter())
jsonOption.WriteIndented <- true

System.IO.File.WriteAllText("sim1sandsch.json", System.Text.Json.JsonSerializer.Serialize(Sim1.sandSch, jsonOption))

In [ ]:
[Sim1.q1; Sim1.q2; Sim1.q3; Sim1.q4; Sim1.q5; Sim1.q6]

index utilization responseTime turnaroundTime cost 0 Item1 Item2 61.7 23.5 Item1 Item2 22.5 6.1 Item1 Item2 86.7 62.8 Item1 Item2 10.86 0 1 Item1 Item2 51.6 7 Item1 Item2 9 1.7 Item1 Item2 74.5 12.7 Item1 Item2 15.73 0 2 Item1 Item2 54 8.5 Item1 Item2 9.7 2.9 Item1 Item2 72.5 14.3 Item1 Item2 14.91 0 3 Item1 Item2 66.5 21.1 Item1 Item2 18.3 7 Item1 Item2 78.3 48.8 Item1 Item2 10.65 0 4 Item1 Item2 66.8 21 Item1 Item2 18.2 7 Item1 Item2 78.6 48.2 Item1 Item2 10.62 0 5 Item1 Item2 73.5 21.2 Item1 Item2 10.3 1.3 Item1 Item2 73.4 63.2 Item1 Item2 9.22 0

In [ ]:
let range = seq { 0.0..0.1..40. }
[
    Sim1.q1 |> SimulatorContextBatchQoS.responseTimeChartData range |> Chart.Spline |> Chart.withTraceName ("No Merge")
    Sim1.q2 |> SimulatorContextBatchQoS.responseTimeChartData range |> Chart.Spline |> Chart.withTraceName ("Static Wait")
    Sim1.q3 |> SimulatorContextBatchQoS.responseTimeChartData range |> Chart.Spline |> Chart.withTraceName ("Random Wait")
    Sim1.q4 |> SimulatorContextBatchQoS.responseTimeChartData range |> Chart.Spline |> Chart.withTraceName ("Dynamic Neutral")
    Sim1.q5 |> SimulatorContextBatchQoS.responseTimeChartData range |> Chart.Spline |> Chart.withTraceName ("Dynamic Context")
    Sim1.q6 |> SimulatorContextBatchQoS.responseTimeChartData range |> Chart.Spline |> Chart.withTraceName ("SAND")
]
|> Chart.combine
|> Chart.withXAxisStyle "زمان پاسخ به ثانیه"

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
let range = seq { 0.0..0.1..200. }
[
    Sim1.q1 |> SimulatorContextBatchQoS.turnaroundTimeChartData range |> Chart.Spline |> Chart.withTraceName ("No Merge")
    Sim1.q2 |> SimulatorContextBatchQoS.turnaroundTimeChartData range |> Chart.Spline |> Chart.withTraceName ("Static Wait")
    Sim1.q3 |> SimulatorContextBatchQoS.turnaroundTimeChartData range |> Chart.Spline |> Chart.withTraceName ("Random Wait")
    Sim1.q4 |> SimulatorContextBatchQoS.turnaroundTimeChartData range |> Chart.Spline |> Chart.withTraceName ("Dynamic Neutral")
    Sim1.q5 |> SimulatorContextBatchQoS.turnaroundTimeChartData range |> Chart.Spline |> Chart.withTraceName ("Dynamic Context")
    Sim1.q6 |> SimulatorContextBatchQoS.turnaroundTimeChartData range |> Chart.Spline |> Chart.withTraceName ("SAND")
]
|> Chart.combine
|> Chart.withXAxisStyle "زمان برگشت به ثانیه"

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
let range = seq { 0.0..0.1..100. }
[
    Sim1.q1 |> SimulatorContextBatchQoS.utilizationChartData range |> Chart.Spline |> Chart.withTraceName ("No Merge")
    Sim1.q2 |> SimulatorContextBatchQoS.utilizationChartData range |> Chart.Spline |> Chart.withTraceName ("Static Wait")
    Sim1.q3 |> SimulatorContextBatchQoS.utilizationChartData range |> Chart.Spline |> Chart.withTraceName ("Random Wait")
    Sim1.q4 |> SimulatorContextBatchQoS.utilizationChartData range |> Chart.Spline |> Chart.withTraceName ("Dynamic Neutral")
    Sim1.q5 |> SimulatorContextBatchQoS.utilizationChartData range |> Chart.Spline |> Chart.withTraceName ("Dynamic Context")
    Sim1.q6 |> SimulatorContextBatchQoS.utilizationChartData range |> Chart.Spline |> Chart.withTraceName ("SAND")
]
|> Chart.combine
|> Chart.withXAxisStyle "بهره‌وری به درصد"

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
let range = seq { 0.0..0.001..4. }
let (c1, _), (c2, _), (c3, _), (c4, _), (c5, _), (c6, _) = Sim1.q1.cost, Sim1.q2.cost, Sim1.q3.cost, Sim1.q4.cost, Sim1.q5.cost, Sim1.q6.cost
let costs = [c1; c2; c3; c4; c5; c6] |> List.map float
let title = ["No Merge"; "Static Wait"; "Random Wait"; "Dynamic Neutral"; "Dynamic Context"; "SAND"]

Chart.Column(costs,title) |> Chart.withTitle "نمودار هزینه"

<!-- Plotly chart will be drawn inside this DIV -->

## for 30 functions

In [ ]:
open SimulationRunnerEngine
open EvaluatorConfigs
open AllSchedulers

module Sim2 =
    let iterationsCount = 4

    let queue = genQueue config_UFD_AW_N 50u
    let runForScheduler = runBatch iterationsCount queue
    //let noMer, staticSc, randSc, dynNeut, dynCtx, sandSch = forAllSchedulers runForScheduler
    //let q1, q2, q3, q4, q5, q6 = qosOf noMer, qosOf staticSc, qosOf randSc, qosOf dynNeut, qosOf dynCtx, qosOf sandSch

    let q1 = { utilization = (69.3, 22.8); responseTime = (14.8<second>, 6.7<second>);  turnaroundTime = (77.3<second>, 60.7<second>); cost = (9.75<usd>, 0.0<usd>)}
    let q2 = { utilization = (50.8, 9.9);  responseTime = (9.3<second>, 1.5<second>);   turnaroundTime = (71.6<second>, 15.9<second>); cost = (15.5<usd>, 0.0<usd>)}
    let q3 = { utilization = (54.5, 10.4); responseTime = (9.36<second>, 1.85<second>); turnaroundTime = (71.9<second>, 17.8<second>); cost = (14.6<usd>, 0.0<usd>)}
    let q4 = { utilization = (72.1, 20.0); responseTime = (12.2<second>, 5.8<second>);  turnaroundTime = (72.6<second>, 47.9<second>); cost = (10.14<usd>, 0.0<usd>)}
    let q5 = { utilization = (72.2, 19.9); responseTime = (13.0<second>, 5.9<second>);  turnaroundTime = (73.1<second>, 48.8<second>); cost = (10.13<usd>, 0.0<usd>)}
    let q6 = { utilization = (84.4, 16.9); responseTime = (9.9<second>, 1.9<second>);   turnaroundTime = (97.2<second>, 61.3<second>); cost = (16.1<usd>, 0.0<usd>)}

In [ ]:
let jsonOption = JsonSerializerOptions()
jsonOption.Converters.Add(JsonFSharpConverter())
jsonOption.WriteIndented <- true

System.IO.File.WriteAllText("sim2sandsch.json", System.Text.Json.JsonSerializer.Serialize(Sim2.sandSch, jsonOption))

In [ ]:
[Sim2.q1; Sim2.q2; Sim2.q3; Sim2.q4; Sim2.q5; Sim2.q6]

index utilization responseTime turnaroundTime cost 0 Item1 Item2 69.3 22.8 Item1 Item2 14.8 6.7 Item1 Item2 77.3 60.7 Item1 Item2 9.75 0 1 Item1 Item2 50.8 9.9 Item1 Item2 9.3 1.5 Item1 Item2 71.6 15.9 Item1 Item2 15.5 0 2 Item1 Item2 54.5 10.4 Item1 Item2 9.36 1.85 Item1 Item2 71.9 17.8 Item1 Item2 14.6 0 3 Item1 Item2 72.1 20 Item1 Item2 12.2 5.8 Item1 Item2 72.6 47.9 Item1 Item2 10.14 0 4 Item1 Item2 72.2 19.9 Item1 Item2 13 5.9 Item1 Item2 73.1 48.8 Item1 Item2 10.13 0 5 Item1 Item2 84.4 16.9 Item1 Item2 9.9 1.9 Item1 Item2 97.2 61.3 Item1 Item2 16.1 0

In [ ]:
let range = seq { 0.0..0.1..30. }
[
    Sim2.q1 |> SimulatorContextBatchQoS.responseTimeChartData range |> Chart.Spline |> Chart.withTraceName ("No Merge")
    Sim2.q2 |> SimulatorContextBatchQoS.responseTimeChartData range |> Chart.Spline |> Chart.withTraceName ("Static Wait")
    Sim2.q3 |> SimulatorContextBatchQoS.responseTimeChartData range |> Chart.Spline |> Chart.withTraceName ("Random Wait")
    Sim2.q4 |> SimulatorContextBatchQoS.responseTimeChartData range |> Chart.Spline |> Chart.withTraceName ("Dynamic Neutral")
    Sim2.q5 |> SimulatorContextBatchQoS.responseTimeChartData range |> Chart.Spline |> Chart.withTraceName ("Dynamic Context")
    Sim2.q6 |> SimulatorContextBatchQoS.responseTimeChartData range |> Chart.Spline |> Chart.withTraceName ("SAND")
]
|> Chart.combine
|> Chart.withXAxisStyle "زمان پاسخ به ثانیه"

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
let range = seq { 0.0..0.1..180. }
[
    Sim2.q1 |> SimulatorContextBatchQoS.turnaroundTimeChartData range |> Chart.Spline |> Chart.withTraceName ("No Merge")
    Sim2.q2 |> SimulatorContextBatchQoS.turnaroundTimeChartData range |> Chart.Spline |> Chart.withTraceName ("Static Wait")
    Sim2.q3 |> SimulatorContextBatchQoS.turnaroundTimeChartData range |> Chart.Spline |> Chart.withTraceName ("Random Wait")
    Sim2.q4 |> SimulatorContextBatchQoS.turnaroundTimeChartData range |> Chart.Spline |> Chart.withTraceName ("Dynamic Neutral")
    Sim2.q5 |> SimulatorContextBatchQoS.turnaroundTimeChartData range |> Chart.Spline |> Chart.withTraceName ("Dynamic Context")
    Sim2.q6 |> SimulatorContextBatchQoS.turnaroundTimeChartData range |> Chart.Spline |> Chart.withTraceName ("SAND")
]
|> Chart.combine
|> Chart.withXAxisStyle "زمان برگشت به ثانیه"

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
let range = seq { 0.0..0.1..100. }
[
    Sim2.q1 |> SimulatorContextBatchQoS.utilizationChartData range |> Chart.Spline |> Chart.withTraceName ("No Merge")
    Sim2.q2 |> SimulatorContextBatchQoS.utilizationChartData range |> Chart.Spline |> Chart.withTraceName ("Static Wait")
    Sim2.q3 |> SimulatorContextBatchQoS.utilizationChartData range |> Chart.Spline |> Chart.withTraceName ("Random Wait")
    Sim2.q4 |> SimulatorContextBatchQoS.utilizationChartData range |> Chart.Spline |> Chart.withTraceName ("Dynamic Neutral")
    Sim2.q5 |> SimulatorContextBatchQoS.utilizationChartData range |> Chart.Spline |> Chart.withTraceName ("Dynamic Context")
    Sim2.q6 |> SimulatorContextBatchQoS.utilizationChartData range |> Chart.Spline |> Chart.withTraceName ("SAND")
]
|> Chart.combine
|> Chart.withXAxisStyle "بهره‌وری به درصد"

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
let range = seq { 0.0..0.001..4. }
let (c1, _), (c2, _), (c3, _), (c4, _), (c5, _), (c6, _) = Sim2.q1.cost, Sim2.q2.cost, Sim2.q3.cost, Sim2.q4.cost, Sim2.q5.cost, Sim2.q6.cost
let costs = [c1; c2; c3; c4; c5; c6] |> List.map float
let title = ["No Merge"; "Static Wait"; "Random Wait"; "Dynamic Neutral"; "Dynamic Context"; "SAND"]

Chart.Column(costs,title) |> Chart.withTitle "هزینه"

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
// Visualization of the General.normalize
// function using Weilbul Stretched Function

let x = [ for x in 0. .. 0.1 .. 20 -> (x, General.normalize x) ]

x |> Chart.Line

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
// Dijestra path finding

let dg = DependencyGraph.init (20u)

(dg.adjMatrix) |> GraphExtensions.printAdjMatrix

let path, distance = dg |> DependencyGraph.dijkestraPath (0u) (5u)
let weights = dg |> DependencyGraph.edgeWeightsFromPath path

path, weights, distance


--------------
|  0 | 909 | 745 | 0 | 637 | 0 | 557 | 0 | 936 | 317 | 209 | 0 | 937 | 0 | 918 | 784 | 0 | 0 | 908 | 453 |
--------------
|  616 | 0 | 787 | 0 | 0 | 781 | 423 | 0 | 0 | 0 | 797 | 0 | 0 | 106 | 738 | 593 | 0 | 49 | 697 | 863 |
--------------
|  344 | 489 | 0 | 198 | 106 | 127 | 0 | 232 | 546 | 0 | 0 | 0 | 960 | 0 | 0 | 220 | 920 | 470 | 0 | 295 |
--------------
|  0 | 0 | 505 | 73 | 0 | 0 | 941 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 722 | 895 | 713 |
--------------
|  426 | 196 | 0 | 763 | 99 | 0 | 0 | 0 | 594 | 601 | 0 | 0 | 0 | 55 | 580 | 0 | 171 | 0 | 0 | 0 |
--------------
|  430 | 0 | 0 | 433 | 994 | 451 | 320 | 767 | 0 | 0 | 0 | 0 | 0 | 596 | 0 | 810 | 325 | 948 | 344 | 819 |
--------------
|  752 | 0 | 0 | 974 | 768 | 809 | 334 | 296 | 0 | 0 | 553 | 955 | 0 | 663 | 237 | 40 | 715 | 54 | 69 | 634 |
--------------
|  492 | 0 | 251 | 0 | 425 | 0 | 0 | 0 | 0 | 0 | 406 | 135 | 681 | 0 | 0 | 60 | 208 | 0 | 825 | 0 |
--------------
|  872 | 831 | 650 | 0 | 985 | 787 | 

In [ ]:
// fsevents
// hiredis
// karma-phantomjs-launcher
// phantomjs-prebuilt
// puppeteer
// serialport
// xml2json
// yeoman-environment
// let dd_impossible = [
//     "fsevents"
//     "hiredis"
//     "karma-phantomjs-launcher"
//     "phantomjs-prebuilt"
//     "puppeteer"
//     "serialport"
//     "xml2json"
//     "yeoman-environment"
// ]
// let dd_1 = PackagesData.dependentUpon.Keys |> List.ofSeq |> List.skipWhile(fun x -> x <> "yeoman-generator")
// PackagesData.downloadAndSaveAllPackageSizes dd_impossible

//dd_1.[1]

In [ ]:
let saturdayData = FrequenciesData.frequencies["saturday"]
Chart.Spline(saturdayData.Keys, saturdayData.Values)

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
open QueueDataGenerator
open QueueFunctionGeneration
open Simulator


let config1 : QueueFunctionGeneration.Options = 
    (IgnoreFrequencyData, IgnoreCoopNetwork, defaultUnawareRandomNormalDepsCount)
let queueData = generateFunctionQueueData config1 6
// let context = runSimulatonWithQueueData "saturday" queueData noMergeScheduler

let x = DependencyGraph(queueData.funcs)


Error: input.fsx (13,15)-(13,32) typecheck error This function takes too many arguments, or is used in a context where a function is not expected

In [ ]:
results |> SimulatorContext.getQoSMeassuresList

index utilization responseTime turnaroundTime cost 0 0.04612887905122879 Days NanosecondOfDay Hours Minutes Seconds Milliseconds SubsecondTicks SubsecondNanoseconds BclCompatibleTicks TotalDays TotalHours TotalMinutes TotalSeconds TotalMilliseconds TotalTicks TotalNanoseconds 0 522000000000 0 8 42 0 0 0 5220000000 0.0060416666666666665 0.145 8.7 522 522000 5220000000 522000000000 Days NanosecondOfDay Hours Minutes Seconds Milliseconds SubsecondTicks SubsecondNanoseconds BclCompatibleTicks TotalDays TotalHours TotalMinutes TotalSeconds TotalMilliseconds TotalTicks TotalNanoseconds 0 528504263147 0 8 48 504 5042631 504263147 5285042631 0.006116947490127315 0.14680673976305555 8.808404385783334 528.504263147 528504.263147 5285042631.47 528504263147 0.0061562505264666665 1 0.06681392075498076 Days NanosecondOfDay Hours Minutes Seconds Milliseconds SubsecondTicks SubsecondNanoseconds BclCompatibleTicks TotalDays TotalHours TotalMinutes TotalSeconds TotalMilliseconds TotalTicks TotalNanoseconds 0 288000000000 0 4 48 0 0 0 2880000000 0.0033333333333333335 0.08 4.8 288 288000 2880000000 288000000000 Days NanosecondOfDay Hours Minutes Seconds Milliseconds SubsecondTicks SubsecondNanoseconds BclCompatibleTicks TotalDays TotalHours TotalMinutes TotalSeconds TotalMilliseconds TotalTicks TotalNanoseconds 0 290116136176 0 4 50 116 1161361 116136176 2901161361 0.0033578256501851853 0.08058781560444445 4.835268936266667 290.116136176 290116.136176 2901161361.76 290116136176 0.003454308519888889 2 0.0935589204955372 Days NanosecondOfDay Hours Minutes Seconds Milliseconds SubsecondTicks SubsecondNanoseconds BclCompatibleTicks TotalDays TotalHours TotalMinutes TotalSeconds TotalMilliseconds TotalTicks TotalNanoseconds 0 343000000000 0 5 43 0 0 0 3430000000 0.003969907407407407 0.09527777777777778 5.716666666666667 343 343000 3430000000 343000000000 Days NanosecondOfDay Hours Minutes Seconds Milliseconds SubsecondTicks SubsecondNanoseconds BclCompatibleTicks TotalDays TotalHours TotalMinutes TotalSeconds TotalMilliseconds TotalTicks TotalNanoseconds 0 347575960282 0 5 47 575 5759602 575960282 3475759602 0.004022869910671296 0.09654887785611112 5.792932671366667 347.575960282 347575.960282 3475759602.82 347575960282 0.004260569386766667 3 0.3125057826459409 Days NanosecondOfDay Hours Minutes Seconds Milliseconds SubsecondTicks SubsecondNanoseconds BclCompatibleTicks TotalDays TotalHours TotalMinutes TotalSeconds TotalMilliseconds TotalTicks TotalNanoseconds 0 262000000000 0 4 22 0 0 0 2620000000 0.0030324074074074073 0.07277777777777777 4.366666666666666 262 262000 2620000000 262000000000 Days NanosecondOfDay Hours Minutes Seconds Milliseconds SubsecondTicks SubsecondNanoseconds BclCompatibleTicks TotalDays TotalHours TotalMinutes TotalSeconds TotalMilliseconds TotalTicks TotalNanoseconds 0 265031932361 0 4 25 31 319323 31932361 2650319323 0.003067499217141204 0.0736199812113889 4.4171988726833336 265.031932361 265031.932361 2650319323.61 265031932361 0.004283380389422223 4 0.27795652702342755 Days NanosecondOfDay Hours Minutes Seconds Milliseconds SubsecondTicks SubsecondNanoseconds BclCompatibleTicks TotalDays TotalHours TotalMinutes TotalSeconds TotalMilliseconds TotalTicks TotalNanoseconds 0 249000000000 0 4 9 0 0 0 2490000000 0.0028819444444444444 0.06916666666666667 4.15 249 249000 2490000000 249000000000 Days NanosecondOfDay Hours Minutes Seconds Milliseconds SubsecondTicks SubsecondNanoseconds BclCompatibleTicks TotalDays TotalHours TotalMinutes TotalSeconds TotalMilliseconds TotalTicks TotalNanoseconds 0 251073821109 0 4 11 73 738211 73821109 2510738211 0.002905947003576389 0.06974272808583333 4.18456368515 251.073821109 251073.821109 2510738211.09 251073821109 0.0038636304154000003 5 0.03665024188370652 Days NanosecondOfDay Hours Minutes Seconds Milliseconds SubsecondTicks SubsecondNanoseconds BclCompatibleTicks TotalDays TotalHours TotalMinutes TotalSeconds TotalMilliseconds TotalTicks TotalNanoseconds 0 701000000000 0 11 41 0 0 0 70100